1. Imports and Setup

In [1]:
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import random
import copy



2. CNN Model Definition

In [2]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()

        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # input: 3x32x32
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                         # output: 32x16x16

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                          # output: 64x8x8
        )

        # Flatten output size: 64 * 8 * 8 = 4096
        self.fc = nn.Sequential(
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)  # flatten before FC
        x = self.fc(x)
        return x

3. Load and Split Among Clients

In [3]:
import os
import shutil
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
import numpy as np
from sklearn.model_selection import train_test_split

# -------------------------------------------------------------------
# ⚙️ Parameters
# -------------------------------------------------------------------
NUM_CLIENTS = 10
ALPHA = 0.1            # smaller = more non-IID
BATCH_SIZE = 32
IMG_SIZE = (128, 128)

# -------------------------------------------------------------------
# 🗂️ Source dataset paths
# -------------------------------------------------------------------
source_au = r"C:\Users\Admin\OneDrive\Desktop\ResearchTrack\CASIA_FL_Project\CASIA 1.0 dataset\CASIA 1.0 dataset\Au\Au"
source_tp = r"C:\Users\Admin\OneDrive\Desktop\ResearchTrack\CASIA_FL_Project\CASIA 1.0 dataset\CASIA 1.0 dataset\Modified Tp\Tp"

DATA_ROOT = r"C:\Users\Admin\OneDrive\Desktop\ResearchTrack\CASIA_FL_Project\data"
auth_dir = os.path.join(DATA_ROOT, "Authentic")
tamp_dir = os.path.join(DATA_ROOT, "Tampered")
os.makedirs(auth_dir, exist_ok=True)
os.makedirs(tamp_dir, exist_ok=True)

# -------------------------------------------------------------------
# 🏗️ Copy files once
# -------------------------------------------------------------------
if len(os.listdir(auth_dir)) == 0:
    print("📦 Copying Authentic images...")
    for file in os.listdir(source_au):
        src = os.path.join(source_au, file)
        dst = os.path.join(auth_dir, file)
        if os.path.isfile(src):
            shutil.copy(src, dst)

if len(os.listdir(tamp_dir)) == 0:
    print("📦 Copying Tampered images...")
    for file in os.listdir(source_tp):
        src = os.path.join(source_tp, file)
        dst = os.path.join(tamp_dir, file)
        if os.path.isfile(src):
            shutil.copy(src, dst)

print("✅ Data organized in ImageFolder structure.\n")

# -------------------------------------------------------------------
# 🧩 Transforms
# -------------------------------------------------------------------
transform = transforms.Compose([
    transforms.Resize(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# -------------------------------------------------------------------
# 📦 Load dataset
# -------------------------------------------------------------------
full_dataset = datasets.ImageFolder(DATA_ROOT, transform=transform)
print(f"✅ Total images: {len(full_dataset)} | Classes: {full_dataset.classes}")

# -------------------------------------------------------------------
# ✂️ Class-wise Train/Test Split (80/20)
# -------------------------------------------------------------------
targets = np.array(full_dataset.targets)
indices = np.arange(len(full_dataset))

train_indices, test_indices = train_test_split(
    indices,
    test_size=0.2,
    stratify=targets,
    random_state=42
)

train_dataset = Subset(full_dataset, train_indices)
test_dataset = Subset(full_dataset, test_indices)

print(f"Train: {len(train_dataset)} | Test: {len(test_dataset)}")

# -------------------------------------------------------------------
# 🧠 Safe Dirichlet non-IID split
# -------------------------------------------------------------------
def noniid_dirichlet_split(dataset, num_clients, alpha):
    """
    Safe Dirichlet non-IID split. Ensures no client has zero samples.
    """
    targets = np.array([dataset.dataset.targets[i] for i in dataset.indices])
    num_classes = len(np.unique(targets))
    class_indices = [np.where(targets == y)[0] for y in range(num_classes)]

    client_indices = [[] for _ in range(num_clients)]

    for c, idx_c in enumerate(class_indices):
        np.random.shuffle(idx_c)
        proportions = np.random.dirichlet(alpha=np.repeat(alpha, num_clients))
        proportions = np.maximum(proportions, 0.001)
        proportions = proportions / proportions.sum()
        split_points = (np.cumsum(proportions) * len(idx_c)).astype(int)[:-1]
        class_split = np.split(idx_c, split_points)

        for i in range(num_clients):
            if len(class_split[i]) > 0:
                client_indices[i].extend(np.array(dataset.indices)[class_split[i]])

    client_datasets = [Subset(dataset.dataset, inds) for inds in client_indices if len(inds) > 0]
    return client_datasets

# -------------------------------------------------------------------
# 🧩 Create client datasets and loaders
# -------------------------------------------------------------------
client_datasets = noniid_dirichlet_split(train_dataset, NUM_CLIENTS, ALPHA)

client_loaders = [
    DataLoader(cd, batch_size=BATCH_SIZE, shuffle=True)
    for cd in client_datasets if len(cd) > 0
]

# -------------------------------------------------------------------
# 🧾 Global test DataLoader
# -------------------------------------------------------------------
testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# -------------------------------------------------------------------
# 📊 Display summary
# -------------------------------------------------------------------
print(f"\n✅ Created {len(client_loaders)} clients (target: {NUM_CLIENTS}) | Dirichlet α={ALPHA}")
for i, cdata in enumerate(client_datasets[:5]):  # print only first 5 clients for brevity
    labels = [full_dataset.targets[idx] for idx in cdata.indices]
    unique, counts = np.unique(labels, return_counts=True)
    label_map = {full_dataset.classes[u]: int(c) for u, c in zip(unique, counts)}
    print(f"Client {i+1}: {len(cdata)} samples | {label_map}")


✅ Data organized in ImageFolder structure.

✅ Total images: 1721 | Classes: ['Authentic', 'Tampered']
Train: 1376 | Test: 345

✅ Created 10 clients (target: 10) | Dirichlet α=0.1
Client 1: 2 samples | {'Tampered': 2}
Client 2: 21 samples | {'Authentic': 5, 'Tampered': 16}
Client 3: 501 samples | {'Authentic': 443, 'Tampered': 58}
Client 4: 37 samples | {'Authentic': 2, 'Tampered': 35}
Client 5: 166 samples | {'Authentic': 155, 'Tampered': 11}


4. Client Update Function

In [4]:
def client_update(model, dataset, epochs, batch_size, lr, testloader=None):
    model = copy.deepcopy(model)
    model.train()
    optimizer = optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for _ in range(epochs):
        for images, labels in loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    # ✅ Optional: Evaluate client model after training
    if testloader:
        model.eval()
        all_preds, all_labels = [], []

        with torch.no_grad():
            for images, labels in testloader:
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        acc = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

        print(f"📍 Client model evaluation after local training:")
        print(f"Accuracy  : {acc:.4f}")
        print(f"Precision : {precision:.4f}")
        print(f"Recall    : {recall:.4f}")
        print(f"F1 Score  : {f1:.4f}")

    return model.state_dict()


5. FedYogi Server Optimizer

In [5]:
def fed_yogi_update(global_model, client_models, delta_prev, v_prev, beta1=0.9, beta2=0.99, eta=0.01, tau=1e-3):
    # Step 1: Compute average client update (delta)
    delta = {}
    for key in global_model.state_dict().keys():
        delta[key] = torch.zeros_like(global_model.state_dict()[key])
        for client_model in client_models:
            delta[key] += client_model[key] - global_model.state_dict()[key]
        delta[key] /= len(client_models)

    # Step 2: Update first moment estimate (momentum)
    for key in delta:
        delta_prev[key] = beta1 * delta_prev[key] + (1 - beta1) * delta[key]

    # Step 3: Update second moment estimate (Yogi-style)
    for key in delta:
        v_prev[key] -= (1 - beta2) * torch.sign(v_prev[key] - delta[key]**2) * delta[key]**2

    # Step 4: Apply adaptive update to global model
    new_state = {}
    for key in global_model.state_dict().keys():
        new_state[key] = global_model.state_dict()[key] + eta * delta_prev[key] / (torch.sqrt(v_prev[key]) + tau)

    global_model.load_state_dict(new_state)
    return global_model, delta_prev, v_prev


6. Evaluation Function

In [6]:
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # ✅ Add these lines here to suppress warnings and handle undefined metrics
    acc = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    print(f"🔍 Evaluation Metrics:")
    print(f"Accuracy  : {acc:.4f}")
    print(f"Precision : {precision:.4f}")
    print(f"Recall    : {recall:.4f}")
    print(f"F1 Score  : {f1:.4f}")



7. Federated Training Loop with Logging

In [7]:
# Hyperparameters
num_clients = 10
clients_per_round = 2
rounds = 100
epochs = 2
batch_size = 32
learning_rate = 0.001

print(f"📌 Hyperparameters:")
print(f"Clients           : {num_clients}")
print(f"Clients per round : {clients_per_round}")
print(f"Rounds            : {rounds}")
print(f"Epochs per client : {epochs}")
print(f"Batch size        : {batch_size}")
print(f"Learning rate     : {learning_rate}")



global_model = SimpleCNN()
delta_prev = {k: torch.zeros_like(v) for k, v in global_model.state_dict().items()}
v_prev = {k: torch.ones_like(v) * 1e-3 for k, v in global_model.state_dict().items()}

for t in range(rounds):
    selected_clients = random.sample(client_datasets, clients_per_round)
    client_models = []

    for client_data in selected_clients:
        updated_weights = client_update(global_model, client_data, epochs, batch_size, learning_rate)
        client_models.append(updated_weights)

    global_model, delta_prev, v_prev = fed_yogi_update(global_model, client_models, delta_prev, v_prev)
    print(f"\n✅ Round {t+1} complete")
    evaluate_model(global_model, testloader)


📌 Hyperparameters:
Clients           : 10
Clients per round : 2
Rounds            : 100
Epochs per client : 2
Batch size        : 32
Learning rate     : 0.001


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x65536 and 4096x128)

In [ ]:
import pandas as pd
import re

# === Step 1: Load your Excel file ===
file_path = r"C:\Users\Admin\OneDrive\Desktop\ResearchTrack\CASIA_FL_Project\models\data\fedyogi_raw_noniid.xlsx"
  # <-- change to your file name

# Read all cells as text (to handle mixed formatting)
df = pd.read_excel(file_path, header=None)

# Combine all cells into one text block (since your Excel might have 1 column or merged cells)
text = " ".join(df.astype(str).fillna("").values.flatten())

# === Step 2: Extract Hyperparameters ===
hyper_pattern = re.compile(
    r"Clients\s*:\s*(\d+).*?"
    r"Clients per round\s*:\s*(\d+).*?"
    r"Rounds\s*:\s*(\d+).*?"
    r"Epochs per client\s*:\s*(\d+).*?"
    r"Batch size\s*:\s*(\d+).*?"
    r"Learning rate\s*:\s*([\d.]+)",
    re.DOTALL
)

hyper = hyper_pattern.search(text)
if hyper:
    clients, cpr, rounds, epochs, batch, lr = hyper.groups()
    base_info = {
        "Clients": int(clients),
        "Clients per round": int(cpr),
        "Total Rounds": int(rounds),
        "Epochs per client": int(epochs),
        "Batch size": int(batch),
        "Learning rate": float(lr)
    }
else:
    base_info = {}

# === Step 3: Extract metrics per round ===
pattern = re.compile(
    r"Round\s*(\d+).*?"
    r"Accuracy\s*:\s*([\d.]+).*?"
    r"Precision\s*:\s*([\d.]+).*?"
    r"Recall\s*:\s*([\d.]+).*?"
    r"F1\s*Score\s*:\s*([\d.]+)",
    re.DOTALL
)

matches = pattern.findall(text)

data = []
for m in matches:
    round_num, acc, prec, rec, f1 = m
    data.append({
        "Round": int(round_num),
        "Accuracy": float(acc),
        "Precision": float(prec),
        "Recall": float(rec),
        "F1 Score": float(f1)
    })

# === Step 4: Create structured DataFrame ===
processed = pd.DataFrame(data)

for k, v in base_info.items():
    processed[k] = v

cols = [
    "Round", "Accuracy", "Precision", "Recall", "F1 Score",
    "Clients", "Clients per round", "Total Rounds",
    "Epochs per client", "Batch size", "Learning rate"
]
processed = processed[cols]

# === Step 5: Save processed output ===
output_path = "processed_fedyogi_results_noniid.xlsx"
processed.to_excel(output_path, index=False)

print(f"✅ Processed {len(processed)} rounds successfully!")
print(f"📊 File saved as: {output_path}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# === Step 1: Load processed Excel file ===
file_path = r"C:\Users\Admin\OneDrive\Desktop\ResearchTrack\CASIA_FL_Project\models\processed_fedyogi_results_noniid.xlsx"  # change if needed
df = pd.read_excel(file_path)

# === Step 2: Check available columns ===
print("✅ Columns found:", df.columns.tolist())

# === Step 3: Create plots ===
metrics = ["Accuracy", "Precision", "Recall", "F1 Score"]

plt.figure(figsize=(12, 8))

for i, metric in enumerate(metrics, 1):
    plt.subplot(2, 2, i)  # 2 rows, 2 columns, subplot index
    plt.plot(df["Round"], df[metric], label=metric, linewidth=1.5)
    plt.xlabel("Round")
    plt.ylabel(metric)
    plt.title(f"{metric} vs Rounds")
    plt.grid(True)
    plt.legend()

plt.tight_layout()
plt.suptitle("Federated Learning Performance Metrics per Round", fontsize=14, y=1.02)
plt.savefig("fedyogi_performance_plots.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# === Step 1: File paths ===
file_noniid = r"C:\Users\Admin\OneDrive\Desktop\ResearchTrack\CASIA_FL_Project\models\processed_fedyogi_results_noniid.xlsx"
file_iid = r"C:\Users\Admin\OneDrive\Desktop\ResearchTrack\Federated Learning\FedYogi\processed_fedyogi_results.xlsx"

# === Step 2: Read both files ===
df_noniid = pd.read_excel(file_noniid)
df_iid = pd.read_excel(file_iid)

# === Step 3: Basic check ===
print("Non-IID columns:", df_noniid.columns.tolist())
print("IID columns:", df_iid.columns.tolist())

# === Step 4: Align both by 'Round' column ===
max_round = min(df_noniid["Round"].max(), df_iid["Round"].max())
df_noniid = df_noniid[df_noniid["Round"] <= max_round]
df_iid = df_iid[df_iid["Round"] <= max_round]

# === Step 5: Define metrics to compare ===
metrics = ["Accuracy", "Precision", "Recall", "F1 Score"]

# === Step 6: Set up white background and plot style ===
plt.style.use("default")
plt.rcParams["axes.facecolor"] = "white"
plt.rcParams["figure.facecolor"] = "white"

plt.figure(figsize=(12, 8))

# === Step 7: Create comparison plots ===
for i, metric in enumerate(metrics, 1):
    plt.subplot(2, 2, i)
    plt.plot(df_iid["Round"], df_iid[metric], color="#007BFF", label="IID Dataset", linewidth=2)        # bright blue
    plt.plot(df_noniid["Round"], df_noniid[metric], color="#FF6600", label="Non-IID Dataset", linewidth=2)  # orange dashed
    plt.xlabel("Round", fontsize=10)
    plt.ylabel(metric, fontsize=10)
    plt.title(f"{metric} vs Rounds", fontsize=11, fontweight="bold")
    plt.grid(True, linestyle=":", alpha=0.7)
    plt.legend(fontsize=9)

plt.tight_layout()
plt.suptitle("FedYogi: IID vs Non-IID Performance Comparison", fontsize=14, fontweight="bold", y=1.03)

# === Step 8: Save before showing ===
plt.savefig("fedyogi_iid_vs_noniid_comparison.png", dpi=300, bbox_inches="tight")

# === Step 9: Display ===
plt.show()
